# COGS 108 - Final Project (change this to your project's title)

## Permissions

Place an `X` in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that student names will be included (but PIDs will be scraped from any groups who include their PIDs).

* [  ] YES - make available
* [  ] NO - keep private

# Overview

*Fill in your overview here*

# Names

- Jared (Ruotian) Chen
- Jimin Cheon
- Kane Gu
- Laurence D'Ercole
- Nisha Davankar

<a id='research_question'></a>
# Research Question

What attributes of a YouTube video affect its popularity in United States?

<a id='background'></a>

## Background & Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


Factors that would potentially **affect** a video's popularity:
- time created: `time`
- duration: `length`
- number of tags: `tags`
- video category: `category`
- video quality: `video.quality`
- ad placement: `has_ad	has_ad_at_beginning`
- popularity of youtuber
    - time youtuber joined youtube: `creator.time`
    - number of followers: `creator.stats.follower`
    - total number of views: `creator.stats.view`
    - total number of videos: `creator.stats.post`

Factors that **determine** a video's long-term popularity:
- like/dislike ratio: `stats.like	stats.dislike`
- number of comments: `stats.comment`
- number of views: `stats.view`

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
import sys

!{sys.executable} -m pip install --quiet --user --upgrade pandas==1.* numpy matplotlib
!{sys.executable} -m pip install --quiet --user --upgrade -r EDA/requirements.txt

In [2]:
from EDA.goodies import *

In [3]:
from EDA.dcollect import restful
from EDA.dcollect import api_youtube as youtube
from EDA.dcollect import api_youtubei as youtubei

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Cleaning

Describe your data cleaning steps here.

In [4]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

'''
dataset_id = 'random'

import string
keyword = '|'.join(string.ascii_letters)



df_search_details = pd.read_pickle(f'EDA/dsamples/youtube_search_{as_fname(keyword)}_{dataset_id}.pkl')

# optional (extra info about channels in our search result)
#df_search_details_channels = pd.read_pickle(f'EDA/dsamples/youtube_search_channels_{as_fname(keyword)}_{dataset_id}.pkl')

'''


df_search_details = pd.read_pickle('EDA/dsamples/youtube_search_random_ascii.pkl')
# optional (extra info about channels in our search result)
#df_search_details_channels = pd.read_pickle(f'EDA/dsamples/youtube_search_channels_random_ascii.pkl')

df_search_details = df_drop_duplicates(df_search_details, level = 'id')
df_report(df_search_details, name = 'Search Results')

"\ndataset_id = 'random'\n\nimport string\nkeyword = '|'.join(string.ascii_letters)\n\n\n\ndf_search_details = pd.read_pickle(f'EDA/dsamples/youtube_search_{as_fname(keyword)}_{dataset_id}.pkl')\n\n# optional (extra info about channels in our search result)\n#df_search_details_channels = pd.read_pickle(f'EDA/dsamples/youtube_search_channels_{as_fname(keyword)}_{dataset_id}.pkl')\n\n"

---

# Results - Search Results

## Data Preview

,title,description,time,length,tags,category,creator.id,stats.like,stats.dislike,stats.comment,stats.view,video.quality,has_ad,has_ad_at_beginning
id,,,,,,,,,,,,,,
bsN96zE8FuE,QAnon 101: The Search for Q,To understand what Q is and why it has been so...,2021-01-25 20:00:06+00:00,0 days 00:46:30,"[vice_videos:premiere, politics, internet, mov...",Entertainment,UCn8zNIfYAQNdrFRrr8oibKw,29754.0,5197.0,15663.0,1653957,HD,False,False
u8Gd9MJsnnE,QAnon: The conspiracy theory spreading fake ne...,Twitter has become the first social media outl...,2020-07-23 23:19:01+00:00,0 days 00:08:59,"[QAnon, conspiracy theory, Donald trump, what ...",News & Politics,UC6o-wWU-v2ClFMwougmK7dA,9614.0,6911.0,11210.0,927526,HD,True,True
Rhum_0F4weU,[NU’EST] Ha99y 9th anniversary ‘Q is.spoiler -...,뉴이스트 9주년 기념 아홉 가지 Q&A!\n9에 진심인 질문과 답변들이 궁금하다면?...,2021-03-10 06:15:00+00:00,0 days 00:01:02,"[뉴이스트, 뉴이스트W, NU'EST, NU'EST W, JR, REN, ARON,...",Music,UCUuyrV8JDv5UAMW2StsL-NA,4350.0,26.0,475.0,14761,HD,True,True
VGrfN3v5JL8,Sitting Down with QAnon Conspiracy Theorists -...,Jim gathers a group of QAnon conspiracy theori...,2018-08-22 01:18:14+00:00,0 days 00:06:21,"[The Jim Jefferies Show, Jim Jefferies, Jim Je...",Comedy,UCUsN5ZwHx2kILm84-jPDeXw,24923.0,9415.0,16787.0,2116763,HD,False,False
w1kebU_p0CA,Julian Assange on Q (AUDIO ONLY),"In a Canadian broadcast exclusive, controversi...",2014-09-29 14:12:57+00:00,0 days 00:35:07,"[cbc, jian ghomeshi, interview, QTV, Julian As...",Entertainment,UC1nw_szfrEsDWcwD32wHE_w,435.0,26.0,144.0,28644,HD,True,True


## Stats

,length,stats.like,stats.dislike,stats.comment,stats.view
count,1282,1.262000e+03,1.262000e+03,1.157000e+03,1.282000e+03
mean,0 days 00:16:22.436037441,3.681238e+05,4.271235e+04,2.272336e+04,5.977500e+07
std,0 days 00:46:51.925637944,1.437965e+06,2.184774e+05,1.691643e+05,2.973773e+08
min,0 days 00:00:00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0 days 00:03:35,3.738250e+03,1.070000e+02,1.170000e+02,1.125988e+05
50%,0 days 00:08:02,1.947850e+04,1.015000e+03,9.500000e+02,1.273258e+06
75%,0 days 00:15:08.750000,1.262668e+05,8.589250e+03,5.361000e+03,1.472842e+07
max,0 days 15:13:24,2.110508e+07,4.142134e+06,5.197883e+06,4.417995e+09


---

In [5]:
assert(youtube.types.topic(1).__eq__('Film & Animation'))


df_search_details_n = df_search_details.copy()
df_search_details_n['category'] = df_search_details_n['category'].astype(str)


df_search_details_n.groupby(['category'])['stats.view'].mean()




category
Autos & Vehicles         6.348438e+06
Comedy                   7.484052e+06
Education                8.788886e+07
Entertainment            4.193423e+07
Film & Animation         7.897365e+07
Gaming                   5.551895e+06
Howto & Style            7.163648e+06
Music                    1.741913e+08
News & Politics          1.066447e+06
Nonprofits & Activism    1.180620e+05
People & Blogs           2.403062e+07
Pets & Animals           1.655519e+08
Science & Technology     1.090702e+07
Sports                   7.576921e+06
Travel & Events          8.855300e+04
Name: stats.view, dtype: float64

In [6]:
df_trending_details = pd.read_pickle('EDA/dsamples/youtube_trending.pkl')

# optional (extra info about channels on the chart)
df_trending_details_channels = pd.read_pickle(f'EDA/dsamples/youtube_trending_channels.pkl')

df_report(df_trending_details, name = 'Trending')

---

# Results - Trending

## Data Preview

,,title,description,time,length,tags,category,stats.like,stats.dislike,stats.comment,stats.view,video.quality,has_ad,has_ad_at_beginning
id,creator.id,,,,,,,,,,,,,
-aToDFwlM1Y,UC2rWFYjCHFcnjNnWhnGvv7Q,CARMEN RUSHED TO THE HOSPITAL BECAUSE OF MISCA...,Carmen rushed to the hospital signs of miscarr...,2021-03-08 22:31:08+00:00,0 days 00:13:20,"[Carmen and Corey, Carmen’s pregnant, Pregnanc...",People & Blogs,30406.0,410.0,4480.0,364643.0,HD,True,True
-hAztPfxZEk,UCtj45MepAoKxZoyR_Mnt86Q,FIRST DAY IN OUR NEW HOME! (HECTIC),LAST VIDEO: \r\n\r\n*MAKE SURE OUR POST NOTIFI...,2021-03-09 03:38:11+00:00,0 days 00:14:23,"[queen Naija, Medicine, Queen, Royalty Squad, ...",Entertainment,25547.0,465.0,1532.0,389765.0,HD,True,True
0L8kRN_bPXM,UC7zOpx9wgvGBCDEjujnAPQA,Big Sean - Deep Reverence ft. Nipsey Hussle,Stream/Download “Detroit 2 https://bigsean.lnk...,2021-03-05 20:00:12+00:00,0 days 00:04:00,"[Big, Sean, Deep, Reverence, Getting, Out, Our...",Music,117003.0,835.0,5298.0,1659213.0,HD,True,True
0PzWnljCpL0,UCvmofFg-oZc4jvBUIfZbjzg,What Is International Women's Day?,International Women’s Day is celebrated every ...,2021-02-09 05:22:52+00:00,0 days 00:06:00,"[international women's day, what is internatio...",Education,2063.0,194.0,646.0,154588.0,HD,True,True
1hlon2py3VU,UCAXEGk-l_ioBMvHa9_uHJjg,Having His Lil Brother Be Mean To Me!,I shouldn't have let it get this far...\n\nJOI...,2021-03-04 00:33:36+00:00,0 days 00:13:21,"[prank, funny, brother, girlfriend, argue, mad...",People & Blogs,32284.0,1377.0,2080.0,934628.0,HD,True,True


## Stats

,length,stats.like,stats.dislike,stats.comment,stats.view
count,217,2.170000e+02,217.000000,209.000000,2.170000e+02
mean,0 days 00:09:55.317972350,1.390709e+05,2353.668203,12995.052632,2.616348e+06
std,0 days 00:09:18.177271140,2.578788e+05,3470.035672,30139.818414,3.823714e+06
min,0 days 00:00:12,5.550000e+02,60.000000,104.000000,8.771000e+04
25%,0 days 00:03:13,1.986400e+04,413.000000,2241.000000,6.863330e+05
50%,0 days 00:07:27,5.341900e+04,926.000000,4493.000000,1.225780e+06
75%,0 days 00:13:20,1.197020e+05,2676.000000,11135.000000,2.910005e+06
max,0 days 00:54:22,1.760165e+06,22133.000000,301036.000000,2.577447e+07


---

In [7]:
#We first reset the index of the dataframe
df_trending_details= df_trending_details.reset_index()

In [8]:
#We clean the unecessary information such as dislikes, creator time, creater stats view and creater stats posts
del df_trending_details['id']
del df_trending_details['creator.id']
del df_trending_details['descriptio']
del df_trending_details['stats.dislike']
del df_trending_details['video.quality']
del df_trending_details['has_ad']

KeyError: 'creator.stats.view'

In [ ]:
#we group the data by if there are ads in the begining and caluculte the mean value of likes, comments and views
has_ad_likes = df_trending_details.groupby(['has_ad_at_beginning'])['stats.like'].mean()
has_ad_comments = df_trending_details.groupby(['has_ad_at_beginning'])['stats.comment'].mean()
has_ad_views = df_trending_details.groupby(['has_ad_at_beginning'])['stats.view'].mean()

has_ad_likes.reset_index()
has_ad_comments.reset_index()
has_ad_views.reset_index()

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION
ax_likes = has_ad_likes.plot.bar(x= 'has_ad_at_beginning', y= 'stats.like', title= "Average likes based on exsistence of ads")
ax_likes.set_ylabel("Average likes")
plt.show()
ax_comments= has_ad_comments.plot.bar(x= 'has_ad_at_beginning', y= 'stats.comment',  title= "Average comments based on exsistence of ads")
ax_comments.set_ylabel("Average comments")
plt.show()
ax_views= has_ad_views.plot.bar(x= 'has_ad_at_beginning', y= 'stats.view', title= "Average views based on exsistence of ads")
ax_views.set_ylabel("Average views")
plt.show()

The plots above compare the vidoes that had an ad at the begining of the vidoe and videos that do not have ads at the begining of a video. The first plot compares has_ad_at_begining to the average likes. The second plot compares has_ad_at_begining to average comments and the final lot compares has_ad_at_begining to average views. We can see that in all three cases, the popularity of the videos is higher when there is no ad attached to the video. 

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*

# Team Contributions

*Specify who in your group worked on which parts of the project.*